# MM-CTR (WWW 2025) — Notebook complet
Ce notebook te donne un pipeline **propre, robuste et extensible** pour augmenter l’AUC sur **Task1 / Task2 / Task1&2**.

✅ Améliorations clés incluses :
- **Padding row (item_id=0) fixé** (vecteur zéro) — indispensable  
- **Teacher séquentiel (Word2Vec *ou* SASRec-lite)** sur `item_seq.parquet`  
- **Distillation image → teacher** avec **Cosine + InfoNCE** (meilleur que MSE)
- **Fusion teacher + student** (gating) + fallback cold-start
- Génération `item_info_updated.parquet` **sans casser la 1ère ligne padding**
- Entraînement CTR via **FuxiCTR** (DIN baseline) + options modèles plus forts (DCNv2 / AutoInt) pour Task1&2

> ⚠️ Règles :  
> - **Task1** : tu modifies seulement `item_emb` dans `item_info.parquet` et tu entraînes **DIN baseline** avec hyperparams limités.  
> - **Task2** : tu dois choisir un des 4 embeddings fournis dans `item_emb.parquet`.  
> - **Task1&2** : tu peux faire **meilleurs embeddings + meilleur modèle CTR**, mais tu dois garder les splits et ne modifier que `item_emb` dans `item_info.parquet`.

---

## 0) Setup
Exécute dans Kaggle.


In [1]:
# =========================
# 0) Setup (Kaggle)
# =========================
!pip -q install -U "transformers>=4.41" datasets polars pyarrow gensim tqdm scikit-learn accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 800.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 MB 41.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 84.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.6.0 requires polars<1.29,>=1.25, but you have polars 1.36.1 which is incompatible.


In [2]:
import os, json, math, time, random
from dataclasses import dataclass
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## 1) Config
> Mets ici tes chemins. (Kaggle : `/kaggle/input/...` et `/kaggle/working/...`)


In [3]:
@dataclass
class CFG:
    # ---------- DATA ----------
    DATA_ROOT: str = "/kaggle/input/www2025-mmctr-data/MicroLens_1M_MMCTR"
    DATASET_ID: str = "MicroLens_1M_x1"
    WORK_DIR: str = "/kaggle/working"

    # raw files (auto)
    ITEM_INFO: str = None
    ITEM_SEQ: str = None
    TRAIN: str = None
    VALID: str = None
    TEST: str = None

    # images (auto)
    IMG_DIR: str = None

    # ---------- EMBEDDINGS ----------
    EMB_DIM: int = 128
    CLIP_MODEL_ID: str = "openai/clip-vit-base-patch32"

    # teacher method: "word2vec" or "sasrec"
    TEACHER_METHOD: str = "word2vec"

    # Word2Vec params
    W2V_WINDOW: int = 10
    W2V_MIN_COUNT: int = 2
    W2V_EPOCHS: int = 5

    # SASRec-lite params
    MAX_LEN: int = 50
    SASREC_EPOCHS: int = 2
    SASREC_BATCH: int = 256
    SASREC_LR: float = 2e-3
    SASREC_NNEG: int = 50

    # student projector params
    PROJ_HIDDEN: int = 256
    PROJ_DROPOUT: float = 0.1
    PROJ_EPOCHS: int = 3
    PROJ_BATCH: int = 512
    PROJ_LR: float = 2e-3
    ALPHA_COS: float = 0.7
    BETA_NCE: float = 0.3
    TEMPERATURE: float = 0.07

    # fusion params
    USE_GATING: bool = True

    # sanity/perf
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    SEED: int = 42

cfg = CFG()

# ----------------------------
# Auto-wire paths (FIXED)
# ----------------------------
cfg.ITEM_INFO = f"{cfg.DATA_ROOT}/{cfg.DATASET_ID}/item_info.parquet"
cfg.TRAIN     = f"{cfg.DATA_ROOT}/{cfg.DATASET_ID}/train.parquet"
cfg.VALID     = f"{cfg.DATA_ROOT}/{cfg.DATASET_ID}/valid.parquet"
cfg.TEST      = f"{cfg.DATA_ROOT}/{cfg.DATASET_ID}/test.parquet"

# IMPORTANT: item_seq est à la racine (pas dans MicroLens_1M_x1)
cfg.ITEM_SEQ  = f"{cfg.DATA_ROOT}/item_seq.parquet"

# IMPORTANT: images = item_images/item_images/
cfg.IMG_DIR   = f"{cfg.DATA_ROOT}/item_images/item_images"

Path(cfg.WORK_DIR).mkdir(parents=True, exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(cfg.SEED)

print("Device:", cfg.DEVICE)
print("ITEM_INFO:", cfg.ITEM_INFO)
print("ITEM_SEQ :", cfg.ITEM_SEQ)
print("IMG_DIR  :", cfg.IMG_DIR)


Device: cuda
ITEM_INFO: /kaggle/input/www2025-mmctr-data/MicroLens_1M_MMCTR/MicroLens_1M_x1/item_info.parquet
ITEM_SEQ : /kaggle/input/www2025-mmctr-data/MicroLens_1M_MMCTR/item_seq.parquet
IMG_DIR  : /kaggle/input/www2025-mmctr-data/MicroLens_1M_MMCTR/item_images/item_images


## 2) Load item_info + vérifier la padding row
La 1ère ligne de `item_info.parquet` est un **padding** et doit être conservée.


In [4]:

item_info = pl.read_parquet(cfg.ITEM_INFO)
print(item_info.head(3))
print("Rows:", item_info.height, "Cols:", item_info.columns)

# Sanity padding
first_item = item_info.row(0)
print("\nFirst row item_id:", first_item[item_info.columns.index("item_id")] if "item_id" in item_info.columns else "N/A")


shape: (3, 3)
┌─────────┬───────────────┬─────────────────────────────────┐
│ item_id ┆ item_tags     ┆ item_emb_d128                   │
│ ---     ┆ ---           ┆ ---                             │
│ i64     ┆ array[i64, 5] ┆ array[f64, 128]                 │
╞═════════╪═══════════════╪═════════════════════════════════╡
│ 0       ┆ [0, 0, … 0]   ┆ [0.0, 0.0, … 0.0]               │
│ 1       ┆ [0, 0, … 1]   ┆ [-0.587725, -0.384628, … -0.05… │
│ 2       ┆ [0, 0, … 4]   ┆ [-2.50544, 1.56058, … 0.154814… │
└─────────┴───────────────┴─────────────────────────────────┘
Rows: 91718 Cols: ['item_id', 'item_tags', 'item_emb_d128']

First row item_id: 0


## 3) Charger les séquences utilisateurs (`item_seq.parquet`)
On essaye d’être robuste : `item_seq` peut être une liste, string JSON, ou une colonne déjà tokenisée.


In [9]:
# =========================================================
# 0) SILENCE WARNINGS + LOGS
# =========================================================
import os, warnings, gc
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
try:
    from transformers.utils import logging as hf_logging
    hf_logging.set_verbosity_error()
except:
    pass

# =========================================================
# 1) IMPORTS
# =========================================================
import numpy as np
import polars as pl
from tqdm.auto import tqdm
from numba import njit

import torch
print("Polars:", pl.__version__)
print("Torch :", torch.__version__)

from sklearn.decomposition import PCA

# =========================================================
# 2) PARAMÈTRES "PLUS FORTS" (AUC)
# =========================================================
SEED   = getattr(cfg, "SEED", 42)
DEVICE = getattr(cfg, "DEVICE", "cuda" if torch.cuda.is_available() else "cpu")

# --- data sampling ---
# On filtre par user_id%KEEP_MOD (RAM-safe), puis on group_by user_id
# Pour +AUC: ↓KEEP_MOD (donc + de data)
KEEP_MOD = 4     # 10=faible, 4=fort, 3=très fort (si RAM OK)
KEEP_REM = 0
DO_GROUPBY_USER = True

# On limite le nombre d'utilisateurs uniques (après group_by) pour éviter RAM explosion
MAX_USERS_W2V = 300_000   # 100k OK, 300k mieux, 500k si Kaggle 30GB

# --- Word2Vec "fort" ---
MAX_LEN_W2V = 200         # Word2Vec apprend mieux avec plus d'historique
W2V_DIM_BIG = 256         # on apprend 256d puis PCA -> 128d
FINAL_DIM = getattr(cfg, "EMB_DIM", 128)

W2V_WINDOW    = 25        # 20-30 souvent bon
W2V_MIN_COUNT = 2         # 2 augmente coverage (rare items)
W2V_EPOCHS    = 12        # 10-15 selon temps
W2V_NEG       = 20        # plus de negatives -> souvent mieux
W2V_SAMPLE    = 1e-5      # moins d'agressivité que 1e-4 (garde info populaires)

# pondération SG/CBOW
ALPHA_SG = 0.6
ALPHA_CBOW = 0.4

np.random.seed(SEED)

# =========================================================
# 3) LOAD item_seq (RAM-SAFE) + group_by user
# =========================================================
def load_user_sequences_hash(item_seq_path: str, keep_mod: int, keep_rem: int, do_groupby: bool = True):
    lazy = (pl.scan_parquet(item_seq_path)
              .select(["user_id", "item_seq"])
              .filter((pl.col("user_id") % keep_mod) == keep_rem))
    df = lazy.collect(streaming=True)

    if do_groupby:
        df = (df.group_by("user_id")
                .agg(pl.col("item_seq").flatten().alias("item_seq")))
    return df

seq_df = load_user_sequences_hash(cfg.ITEM_SEQ, KEEP_MOD, KEEP_REM, DO_GROUPBY_USER)
print("✅ after hash+group_by:", seq_df.shape)

# limiter le nb d'utilisateurs uniques (pour RAM/temps)
if seq_df.height > MAX_USERS_W2V:
    seq_df = seq_df.sample(n=MAX_USERS_W2V, seed=SEED)
    print("✅ after cap MAX_USERS_W2V:", seq_df.shape)

# Conversion (sur un df limité)
seqs = seq_df["item_seq"].to_list()
del seq_df
gc.collect()
print("✅ nb users(seqs):", len(seqs))

# =========================================================
# 4) PACK en matrice X (N, MAX_LEN_W2V) + Numba clean
# =========================================================
X = np.zeros((len(seqs), MAX_LEN_W2V), dtype=np.int32)

for i, s in enumerate(tqdm(seqs, desc="Packing X (W2V)")):
    # remove 0, keep >0
    tmp = [int(v) for v in s if v and int(v) > 0]
    if len(tmp) < 2:
        continue
    tmp = tmp[-MAX_LEN_W2V:]
    X[i, -len(tmp):] = np.array(tmp, dtype=np.int32)

del seqs
gc.collect()

@njit
def compute_valid(X):
    n, L = X.shape
    valid = np.zeros(n, dtype=np.uint8)
    for i in range(n):
        cnt = 0
        for j in range(L):
            if X[i, j] != 0:
                cnt += 1
        if cnt >= 2:
            valid[i] = 1
    return valid

@njit
def compact_rows(X, valid):
    n, L = X.shape
    m = 0
    for i in range(n):
        if valid[i] == 1:
            m += 1
    Y = np.zeros((m, L), dtype=X.dtype)
    k = 0
    for i in range(n):
        if valid[i] == 1:
            for j in range(L):
                Y[k, j] = X[i, j]
            k += 1
    return Y

valid = compute_valid(X)
X = compact_rows(X, valid)
del valid
gc.collect()
print("✅ X final:", X.shape)

Polars: 1.36.1
Torch : 2.8.0+cu126
✅ after hash+group_by: (250000, 2)
✅ nb users(seqs): 250000


Packing X (W2V):   0%|          | 0/250000 [00:00<?, ?it/s]

✅ X final: (250000, 200)


# 4) Teacher embeddings (séquences → 128d)
Deux choix :
- **Word2Vec** (rapide, baseline)
- **SASRec-lite** (transformer séquentiel, souvent meilleur pour AUC)

Choisis via `cfg.TEACHER_METHOD == Word2Vec`.

In [10]:
# =========================================================
# 5) Word2Vec++ : SG + CBOW en 256d (iterator RAM-safe)
# =========================================================
class XSentenceIterator:
    def __init__(self, X):
        self.X = X
    def __iter__(self):
        X = self.X
        for i in range(X.shape[0]):
            row = X[i]
            tokens = [str(int(v)) for v in row if v != 0]
            if len(tokens) >= 2:
                yield tokens

def train_w2v(X, vector_size=256, sg=1):
    from gensim.models import Word2Vec
    it = XSentenceIterator(X)

    model = Word2Vec(
        vector_size=vector_size,
        window=W2V_WINDOW,
        min_count=W2V_MIN_COUNT,
        workers=4,
        sg=sg,                 # 1=skip-gram, 0=cbow
        negative=W2V_NEG,
        sample=W2V_SAMPLE,
        epochs=W2V_EPOCHS,
        seed=SEED
    )
    print(f"➡️ build_vocab (sg={sg}) ...")
    model.build_vocab(it)

    print(f"➡️ train (sg={sg}) ...")
    model.train(it, total_examples=model.corpus_count, epochs=model.epochs)

    kv = model.wv
    out = {int(k): kv[k].astype(np.float32) for k in kv.index_to_key}
    del model
    gc.collect()
    return out

teacher_sg = train_w2v(X, vector_size=W2V_DIM_BIG, sg=1)
teacher_cb = train_w2v(X, vector_size=W2V_DIM_BIG, sg=0)

# =========================================================
# 6) Combiner SG/CBOW puis PCA -> 128d
# =========================================================
# union des items
all_keys = sorted(set(teacher_sg.keys()) | set(teacher_cb.keys()))
print("✅ union vocab:", len(all_keys))

# matrice [V, 256]
M = np.zeros((len(all_keys), W2V_DIM_BIG), dtype=np.float32)
for i, k in enumerate(tqdm(all_keys, desc="Fusion SG/CBOW")):
    v = None
    if k in teacher_sg and k in teacher_cb:
        v = ALPHA_SG * teacher_sg[k] + ALPHA_CBOW * teacher_cb[k]
    elif k in teacher_sg:
        v = teacher_sg[k]
    else:
        v = teacher_cb[k]
    M[i] = v

del teacher_sg, teacher_cb
gc.collect()

print("➡️ PCA 256 ->", FINAL_DIM)
pca = PCA(n_components=FINAL_DIM, svd_solver="randomized", random_state=SEED)
M128 = pca.fit_transform(M).astype(np.float32)
del M
gc.collect()

teacher = {k: M128[i] for i, k in enumerate(all_keys)}
del M128, all_keys
gc.collect()

# =========================================================
# 7) Remplir les items manquants via item_tags (améliore coverage)
# =========================================================
item_info_small = (pl.scan_parquet(cfg.ITEM_INFO)
                     .select(["item_id", "item_tags"])
                     .collect(streaming=True))
item_ids = item_info_small["item_id"].to_list()
item_tags = item_info_small["item_tags"].to_list()
del item_info_small
gc.collect()

# build tag -> (sum,count) à partir des items déjà appris
tag_sum = {}
tag_cnt = {}
global_sum = np.zeros((FINAL_DIM,), dtype=np.float32)
global_cnt = 0

for iid, tags in tqdm(zip(item_ids, item_tags), total=len(item_ids), desc="Tag stats"):
    iid = int(iid)
    if iid == 0:
        continue
    v = teacher.get(iid, None)
    if v is None:
        continue
    global_sum += v
    global_cnt += 1
    for t in tags:
        t = int(t)
        if t == 0:
            continue
        if t not in tag_sum:
            tag_sum[t] = v.copy()
            tag_cnt[t] = 1
        else:
            tag_sum[t] += v
            tag_cnt[t] += 1

global_mean = global_sum / max(global_cnt, 1)

# impute missing
missing = 0
for iid, tags in tqdm(zip(item_ids, item_tags), total=len(item_ids), desc="Impute missing"):
    iid = int(iid)
    if iid == 0:
        continue
    if iid in teacher:
        continue

    vecs = []
    for t in tags:
        t = int(t)
        if t != 0 and t in tag_sum:
            vecs.append(tag_sum[t] / tag_cnt[t])

    if len(vecs) > 0:
        v = np.mean(np.stack(vecs, axis=0), axis=0).astype(np.float32)
        # shrinkage vers global mean (évite bruit tags rares)
        v = 0.8 * v + 0.2 * global_mean
    else:
        # fallback global_mean + petit bruit
        v = global_mean + (np.random.normal(0, 0.01, size=(FINAL_DIM,)).astype(np.float32))

    teacher[iid] = v
    missing += 1

print("✅ missing imputés:", missing)

# =========================================================
# 8) Normalisation L2 (souvent meilleur pour CTR)
# =========================================================
def l2_normalize(v, eps=1e-12):
    n = float(np.sqrt((v * v).sum()))
    if n < eps:
        return v
    return (v / n).astype(np.float32)

for k in list(teacher.keys()):
    teacher[k] = l2_normalize(teacher[k])

# =========================================================
# 9) Construire matrice finale item_emb (n_items, 128)
# =========================================================
max_item_id = (pl.scan_parquet(cfg.ITEM_INFO)
                 .select(pl.col("item_id").max().alias("m"))
                 .collect(streaming=True)["m"][0])
n_items = int(max_item_id) + 1

emb_final = np.zeros((n_items, FINAL_DIM), dtype=np.float32)
# padding row 0 = zeros (important)
for i in range(1, n_items):
    if i in teacher:
        emb_final[i] = teacher[i]
    else:
        # rarement
        emb_final[i] = l2_normalize(global_mean.copy())

coverage = (np.count_nonzero(np.linalg.norm(emb_final[1:], axis=1) > 0) / (n_items - 1))
print("✅ n_items:", n_items)
print("✅ coverage final:", coverage)

➡️ build_vocab (sg=1) ...
➡️ train (sg=1) ...
➡️ build_vocab (sg=0) ...
➡️ train (sg=0) ...
✅ union vocab: 88688


Fusion SG/CBOW:   0%|          | 0/88688 [00:00<?, ?it/s]

➡️ PCA 256 -> 128


Tag stats:   0%|          | 0/91718 [00:00<?, ?it/s]

Impute missing:   0%|          | 0/91718 [00:00<?, ?it/s]

✅ missing imputés: 3029
✅ n_items: 91718
✅ coverage final: 1.0


In [11]:
import polars as pl
import numpy as np
import gc

# Lire item_info
item_info = pl.read_parquet(cfg.ITEM_INFO)
print("item_info:", item_info.shape, item_info.columns)
print(item_info.head(2))

# emb_final: numpy [n_items, 128] float32
assert emb_final.shape[0] == item_info.height, "n_items mismatch (doit inclure la ligne 0 padding)"
assert emb_final.shape[1] == 128, "dim mismatch"

# Convertir en list[list[float]] pour Polars (col array/list)
# ⚠️ pour éviter surcoût RAM: conversion en float32 + tolist()
emb_list = emb_final.astype(np.float32).tolist()

# Remplacer la colonne
item_info_new = item_info.with_columns(
    pl.Series("item_emb_d128", emb_list)
)

# Sauvegarder (dans working)
out_path = f"{cfg.WORK_DIR}/item_info.parquet"
item_info_new.write_parquet(out_path)
print("✅ Saved:", out_path)

del item_info, item_info_new, emb_list
gc.collect()

item_info: (91718, 3) ['item_id', 'item_tags', 'item_emb_d128']
shape: (2, 3)
┌─────────┬───────────────┬─────────────────────────────────┐
│ item_id ┆ item_tags     ┆ item_emb_d128                   │
│ ---     ┆ ---           ┆ ---                             │
│ i64     ┆ array[i64, 5] ┆ array[f64, 128]                 │
╞═════════╪═══════════════╪═════════════════════════════════╡
│ 0       ┆ [0, 0, … 0]   ┆ [0.0, 0.0, … 0.0]               │
│ 1       ┆ [0, 0, … 1]   ┆ [-0.587725, -0.384628, … -0.05… │
└─────────┴───────────────┴─────────────────────────────────┘
✅ Saved: /kaggle/working/item_info.parquet


23

In [12]:
tmp = pl.read_parquet(f"{cfg.WORK_DIR}/item_info.parquet")
print(tmp.head(2))
print("Shape:", tmp.shape)
print("Type col item_emb_d128:", tmp.schema["item_emb_d128"])

shape: (2, 3)
┌─────────┬───────────────┬─────────────────────────────────┐
│ item_id ┆ item_tags     ┆ item_emb_d128                   │
│ ---     ┆ ---           ┆ ---                             │
│ i64     ┆ array[i64, 5] ┆ list[f64]                       │
╞═════════╪═══════════════╪═════════════════════════════════╡
│ 0       ┆ [0, 0, … 0]   ┆ [0.0, 0.0, … 0.0]               │
│ 1       ┆ [0, 0, … 1]   ┆ [-0.289416, 0.232379, … 0.1086… │
└─────────┴───────────────┴─────────────────────────────────┘
Shape: (91718, 3)
Type col item_emb_d128: List(Float64)


# 5) CLIP image embeddings (512d)
On extrait des features image CLIP (512d), puis on distille vers 128d.


In [14]:
# =========================================================
# 0) SILENCE WARNINGS + TF/XLA NOISE
# =========================================================
import os, warnings, gc
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # réduit bruit TF/XLA si jamais présent

# =========================================================
# 1) IMPORTS
# =========================================================
import numpy as np
import polars as pl
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from transformers import CLIPModel, CLIPImageProcessor

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True

print("Device:", cfg.DEVICE)

# =========================================================
# 2) LOAD item_info (FIX NameError)
# =========================================================
item_info = pl.read_parquet(cfg.ITEM_INFO)  # ou cfg.WORK_DIR + "/item_info.parquet"
print("✅ item_info:", item_info.shape, item_info.columns)

# =========================================================
# 3) DATASET
# =========================================================
class ImgPathDataset(Dataset):
    def __init__(self, item_ids, img_dir):
        self.item_ids = [int(i) for i in item_ids]
        self.img_dir = Path(img_dir)

    def __len__(self):
        return len(self.item_ids)

    def __getitem__(self, idx):
        iid = self.item_ids[idx]
        return iid, str(self.img_dir / f"{iid}.jpg")

def _safe_open_rgb(path, size=(224,224)):
    try:
        return Image.open(path).convert("RGB")
    except Exception:
        return Image.fromarray(np.zeros((size[0], size[1], 3), dtype=np.uint8))

def make_collate(processor: CLIPImageProcessor):
    def collate(batch):
        ids = [b[0] for b in batch]
        paths = [b[1] for b in batch]
        imgs = [_safe_open_rgb(p) for p in paths]
        inputs = processor(images=imgs, return_tensors="pt")
        return torch.tensor(ids, dtype=torch.long), inputs["pixel_values"]
    return collate

# =========================================================
# 4) EXTRACT (memmap)
# =========================================================
def extract_clip_features_memmap(
    item_ids,
    img_dir,
    model_id="openai/clip-vit-base-patch32",
    out_memmap_path="/kaggle/working/clip_img_feats.f16.mmap",
    batch_size=256,
    num_workers=4,
    use_projected_512=True,
    device=None
):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    processor = CLIPImageProcessor.from_pretrained(model_id)
    model = CLIPModel.from_pretrained(model_id).to(device).eval()

    ds = ImgPathDataset(item_ids, img_dir)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=(num_workers > 0),
        collate_fn=make_collate(processor),
        prefetch_factor=2 if num_workers > 0 else None
    )

    feat_dim = model.config.projection_dim if use_projected_512 else model.vision_model.config.hidden_size

    feats = np.memmap(out_memmap_path, mode="w+", dtype=np.float16, shape=(len(ds), feat_dim))
    ids_out = np.zeros((len(ds),), dtype=np.int32)

    idx0 = 0
    pbar = tqdm(dl, desc=f"CLIP extract ({feat_dim}d)")
    with torch.inference_mode():
        for ids, px in pbar:
            bs = ids.shape[0]
            px = px.to(device, non_blocking=True)

            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=str(device).startswith("cuda")):
                if use_projected_512:
                    out = model.get_image_features(pixel_values=px)  # [B,512]
                else:
                    out = model.vision_model(pixel_values=px).last_hidden_state[:, 0, :]  # [B,768]
                out = F.normalize(out, dim=-1)

            out = out.detach().cpu().numpy().astype(np.float16)
            feats[idx0:idx0+bs] = out
            ids_out[idx0:idx0+bs] = ids.numpy().astype(np.int32)
            idx0 += bs

    feats.flush()
    del model, processor, ds, dl
    gc.collect()
    if str(device).startswith("cuda"):
        torch.cuda.empty_cache()

    return out_memmap_path, ids_out, feat_dim

# =========================================================
# 5) RUN
# =========================================================
item_ids_all = item_info["item_id"].to_list()  # ✅ maintenant item_info existe

mmap_path, clip_ids, clip_dim = extract_clip_features_memmap(
    item_ids=item_ids_all,
    img_dir=cfg.IMG_DIR,
    model_id=cfg.CLIP_MODEL_ID,
    out_memmap_path=f"{cfg.WORK_DIR}/clip_img_feats.f16.mmap",
    batch_size=256,     # 128 si OOM
    num_workers=4,
    use_projected_512=True,
    device=cfg.DEVICE
)

print("✅ CLIP dim:", clip_dim)
print("✅ saved memmap:", mmap_path)
print("✅ ids:", clip_ids.shape, "example:", clip_ids[:5])

Device: cuda
✅ item_info: (91718, 3) ['item_id', 'item_tags', 'item_emb_d128']


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

CLIP extract (512d):   0%|          | 0/359 [00:00<?, ?it/s]

✅ CLIP dim: 512
✅ saved memmap: /kaggle/working/clip_img_feats.f16.mmap
✅ ids: (91718,) example: [0 1 2 3 4]


# 6) Student projector (CLIP → 128d) avec **Cosine + InfoNCE**
On aligne les items qui ont à la fois :
- CLIP feature
- teacher embedding

Puis on entraîne un projecteur MLP.

**Loss :**
- `L_cos = 1 - cosine(student, teacher)`
- `L_nce = cross_entropy(sim(student, teacher)/T)` (diagonal positives in-batch)

> Cette distillation marche souvent mieux que MSE pour la recommandation.


In [15]:
# =========================================================
# 0) IMPORTS + FLAGS
# =========================================================
import os, gc
import numpy as np
import polars as pl
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True

DEVICE = cfg.DEVICE
SEED   = getattr(cfg, "SEED", 42)

def seed_everything(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(SEED)

# =========================================================
# 1) LOAD TEACHER (128d) + item_ids
# =========================================================
item_info = pl.read_parquet(f"{cfg.WORK_DIR}/item_info.parquet")  # ou cfg.ITEM_INFO
item_ids = item_info["item_id"].to_list()
n_items  = item_info.height

teacher128 = np.vstack([np.array(v, dtype=np.float32) for v in item_info["item_emb_d128"].to_list()])
assert teacher128.shape == (n_items, 128)

# Option : ignorer teacher quasi-zero (si coverage < 1.0)
teacher_norm = np.linalg.norm(teacher128, axis=1)
valid_teacher = teacher_norm > 1e-8

print("✅ n_items:", n_items)
print("✅ teacher128:", teacher128.shape, "valid_teacher:", int(valid_teacher.sum()))

# =========================================================
# 2) LOAD CLIP MEMMAP (512d)
# =========================================================
clip_dim = 512
clip_path = f"{cfg.WORK_DIR}/clip_img_feats.f16.mmap"

clip_mem = np.memmap(clip_path, mode="r", dtype=np.float16, shape=(n_items, clip_dim))
# On ne cast PAS tout en float32 ici (sinon RAM). On cast par batch dans __getitem__.

print("✅ clip memmap ok:", clip_path)

# =========================================================
# 3) BUILD TRAIN INDEXES (aligned pairs)
# =========================================================
item_id_arr = np.array(item_ids, dtype=np.int64)
valid = (item_id_arr != 0) & valid_teacher  # padding row 0 out

train_idx = np.where(valid)[0].astype(np.int32)
print("✅ aligned pairs:", len(train_idx))

# Option (rapide) : sous-échantillonner si tu veux aller plus vite
# ex: garder 60k exemples
MAX_PAIRS = getattr(cfg, "PROJ_MAX_PAIRS", 0)  # 0 => tout
if MAX_PAIRS and len(train_idx) > MAX_PAIRS:
    rng = np.random.default_rng(SEED)
    train_idx = rng.choice(train_idx, size=MAX_PAIRS, replace=False).astype(np.int32)
    print("✅ sampled pairs:", len(train_idx))

# =========================================================
# 4) DATASET (memmap-safe)
# =========================================================
class PairIndexDataset(Dataset):
    def __init__(self, indices, clip_mem, teacher128):
        self.indices = indices
        self.clip_mem = clip_mem
        self.teacher = teacher128

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        idx = int(self.indices[i])
        x = np.array(self.clip_mem[idx], dtype=np.float32)  # [512] cast local
        y = self.teacher[idx].astype(np.float32)           # [128]
        return torch.from_numpy(x), torch.from_numpy(y)

ds = PairIndexDataset(train_idx, clip_mem, teacher128)

dl = DataLoader(
    ds,
    batch_size=cfg.PROJ_BATCH,
    shuffle=True,
    num_workers=0,          # ✅ le plus stable avec memmap (tu peux tenter 2 si OK)
    pin_memory=True,
    drop_last=True
)

# =========================================================
# 5) PROJECTOR MLP
# =========================================================
class Projector(nn.Module):
    def __init__(self, in_dim, out_dim=128, hidden=256, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, out_dim),
        )
    def forward(self, x):
        x = self.net(x)
        x = F.normalize(x, dim=-1)
        return x

proj = Projector(
    in_dim=clip_dim,
    out_dim=cfg.EMB_DIM,
    hidden=cfg.PROJ_HIDDEN,
    dropout=cfg.PROJ_DROPOUT
).to(DEVICE)

opt = torch.optim.AdamW(proj.parameters(), lr=cfg.PROJ_LR, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler(enabled=str(DEVICE).startswith("cuda"))

# (option) petit scheduler simple
total_steps = cfg.PROJ_EPOCHS * len(dl)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=max(1, total_steps))

# =========================================================
# 6) TRAIN (Cosine + InfoNCE symétrique)
# =========================================================
T = float(cfg.TEMPERATURE)
alpha = float(cfg.ALPHA_COS)
beta  = float(cfg.BETA_NCE)

for ep in range(cfg.PROJ_EPOCHS):
    proj.train()
    pbar = tqdm(dl, desc=f"Projector epoch {ep+1}/{cfg.PROJ_EPOCHS}")
    run_loss = 0.0

    for step, (x, y) in enumerate(pbar):
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)
        y = F.normalize(y, dim=-1)

        opt.zero_grad(set_to_none=True)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=str(DEVICE).startswith("cuda")):
            z = proj(x)  # [B,128]

            # cosine loss
            cos = (z * y).sum(-1)              # [B]
            loss_cos = (1.0 - cos).mean()

            # InfoNCE in-batch (z vs y)
            sim_zy = (z @ y.t()) / T           # [B,B]
            labels = torch.arange(sim_zy.size(0), device=DEVICE)
            loss_nce_1 = F.cross_entropy(sim_zy, labels)

            # InfoNCE symétrique (y vs z) -> souvent un petit gain
            sim_yz = (y @ z.t()) / T
            loss_nce_2 = F.cross_entropy(sim_yz, labels)

            loss_nce = 0.5 * (loss_nce_1 + loss_nce_2)

            loss = alpha * loss_cos + beta * loss_nce

        scaler.scale(loss).backward()
        scaler.unscale_(opt)
        nn.utils.clip_grad_norm_(proj.parameters(), 1.0)
        scaler.step(opt)
        scaler.update()
        sched.step()

        run_loss += float(loss.item())
        pbar.set_postfix(
            loss=run_loss/(step+1),
            cos=float(cos.mean().item()),
            nce=float(loss_nce.item()),
            lr=float(opt.param_groups[0]["lr"])
        )

print("✅ Projector trained.")

✅ n_items: 91718
✅ teacher128: (91718, 128) valid_teacher: 91717
✅ clip memmap ok: /kaggle/working/clip_img_feats.f16.mmap
✅ aligned pairs: 91717


Projector epoch 1/3:   0%|          | 0/179 [00:00<?, ?it/s]

Projector epoch 2/3:   0%|          | 0/179 [00:00<?, ?it/s]

Projector epoch 3/3:   0%|          | 0/179 [00:00<?, ?it/s]

✅ Projector trained.


# 7) Construire `item_emb_d128` final (fusion teacher + student)
- Si item a teacher : fusion (gate) **ou** moyenne
- Sinon : student (cold-start)
- `item_id=0` => vecteur **zéro**


In [16]:
import numpy as np
import polars as pl
import gc
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

DEVICE = cfg.DEVICE
EMB_DIM = cfg.EMB_DIM
clip_dim = 512

# =========================
# 1) Reload item_info + teacher128
# =========================
item_info = pl.read_parquet(f"{cfg.WORK_DIR}/item_info.parquet")
item_ids = np.array(item_info["item_id"].to_list(), dtype=np.int64)
n_items = item_info.height

teacher128 = np.vstack([np.array(v, dtype=np.float32) for v in item_info["item_emb_d128"].to_list()])
teacher128[0] = 0.0  # padding
teacher128 = teacher128.astype(np.float32)

# normalize teacher
t_norm = np.linalg.norm(teacher128, axis=1, keepdims=True) + 1e-12
teacherN = teacher128 / t_norm

print("✅ n_items:", n_items, "teacher128:", teacher128.shape)

# =========================
# 2) Load CLIP memmap (float16)
# =========================
clip_mem = np.memmap(
    f"{cfg.WORK_DIR}/clip_img_feats.f16.mmap",
    mode="r", dtype=np.float16, shape=(n_items, clip_dim)
)

# =========================
# 3) Fonction gating (cos-based) en batch
# g = clamp((cos+1)/2)  -> stable et simple
# final = g*teacher + (1-g)*student   (ou l'inverse si tu veux)
# =========================
@torch.no_grad()
def build_final_embeddings_from_memmap(
    proj,
    clip_mem,
    teacherN,
    batch_size=4096
):
    proj.eval()

    final = np.zeros((n_items, EMB_DIM), dtype=np.float32)

    # item 0 = zeros
    final[0] = 0.0

    pbar = tqdm(range(1, n_items, batch_size), desc="Infer+Fuse (memmap)")
    for i0 in pbar:
        i1 = min(n_items, i0 + batch_size)

        # ---- CLIP batch -> torch
        x = np.array(clip_mem[i0:i1], dtype=np.float32)  # cast local
        x = torch.from_numpy(x).to(DEVICE, non_blocking=True)

        # ---- student
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=str(DEVICE).startswith("cuda")):
            s = proj(x)  # [B,128] déjà normalized

        # ---- teacher normalized batch
        t = torch.from_numpy(teacherN[i0:i1]).to(DEVICE, non_blocking=True)  # [B,128]

        # ---- cosine + gate
        cos = (t * s).sum(-1)                         # [-1,1]
        g = torch.clamp((cos + 1.0) * 0.5, 0.0, 1.0)  # [0,1]

        # ---- fusion
        # (tu peux inverser si tu veux favoriser student)
        v = g.unsqueeze(-1) * t + (1.0 - g).unsqueeze(-1) * s
        v = F.normalize(v, dim=-1)

        final[i0:i1] = v.detach().cpu().numpy().astype(np.float32)

    return final

final128 = build_final_embeddings_from_memmap(
    proj=proj,
    clip_mem=clip_mem,
    teacherN=teacherN,
    batch_size=4096  # 2048/4096/8192 selon GPU
)

print("✅ final128:", final128.shape, "norm[1]:", np.linalg.norm(final128[1]))

# =========================
# 4) Save item_info.parquet final
# =========================
out_path = f"{cfg.WORK_DIR}/item_info.parquet"
item_info_new = item_info.with_columns(
    pl.Series("item_emb_d128", final128.tolist())
)
item_info_new.write_parquet(out_path)

print("✅ Saved final item_info:", out_path)

# cleanup
del item_info_new, item_info, teacher128, teacherN, final128
gc.collect()
if str(DEVICE).startswith("cuda"):
    torch.cuda.empty_cache()

✅ n_items: 91718 teacher128: (91718, 128)


Infer+Fuse (memmap):   0%|          | 0/23 [00:00<?, ?it/s]

✅ final128: (91718, 128) norm[1]: 0.99999994
✅ Saved final item_info: /kaggle/working/item_info.parquet


# 8) Écrire `item_info_updated.parquet` (IMPORTANT)
- On garde toutes les colonnes
- On remplace **uniquement** `item_emb` (ou on crée `item_emb`)
- On garde la première ligne padding (item_id=0)


In [18]:
import polars as pl
import numpy as np

df = pl.read_parquet("/kaggle/working/item_info.parquet")

final128 = np.vstack([np.array(v, dtype=np.float32) for v in df["item_emb_d128"].to_list()])
print("final128:", final128.shape)

# sécurité padding
if int(df["item_id"][0]) == 0:
    final128[0] = 0.0

df_upd = df.with_columns(pl.Series("item_emb", final128.tolist()))

out_path = "/kaggle/working/item_info_updated.parquet"
df_upd.write_parquet(out_path)

print("✅ Saved:", out_path)
print(df_upd.select(["item_id","item_emb"]).head(3))

final128: (91718, 128)
✅ Saved: /kaggle/working/item_info_updated.parquet
shape: (3, 2)
┌─────────┬─────────────────────────────────┐
│ item_id ┆ item_emb                        │
│ ---     ┆ ---                             │
│ i64     ┆ list[f64]                       │
╞═════════╪═════════════════════════════════╡
│ 0       ┆ [0.0, 0.0, … 0.0]               │
│ 1       ┆ [-0.159443, 0.273246, … 0.0841… │
│ 2       ┆ [-0.221092, 0.021761, … 0.0078… │
└─────────┴─────────────────────────────────┘


# 9) CTR training & inference (FuxiCTR baseline + modèles plus forts)
Deux modes :
- **Task1 compliant** : DIN baseline + hyperparams limités
- **Task1&2** : tu peux tester des modèles plus forts : **DCNv2**, **AutoInt**, etc.

On clone le repo officiel et on prépare les fichiers.


In [25]:
# =========================================================
# 9.0) CLONE PROPRE + INSTALL + PREP DATA (ROBUST)
# =========================================================
from pathlib import Path
import shutil, os, gc
import polars as pl
import numpy as np

WORK_DIR = Path("/kaggle/working")

# 1) Ton dossier data déjà prêt (d'après ton screenshot)
DATA_READY_DIR = WORK_DIR / "WWW2025_MMCTR_Challenge" / "data" / cfg.DATASET_ID
assert DATA_READY_DIR.exists(), f"Data dir introuvable: {DATA_READY_DIR}"

print("✅ DATA_READY_DIR:", DATA_READY_DIR)
print("   files:", [p.name for p in DATA_READY_DIR.iterdir()])

# 2) Clone dans un autre dossier (pour ne pas écraser tes fichiers)
REPO_CLONE_DIR = WORK_DIR / "WWW2025_MMCTR_Challenge_repo"

if REPO_CLONE_DIR.exists():
    print("ℹ️ Repo clone dir existe déjà:", REPO_CLONE_DIR)
else:
    !git clone -q --depth 1 https://github.com/reczoo/WWW2025_MMCTR_Challenge.git {str(REPO_CLONE_DIR)}
    print("✅ Repo cloné dans:", REPO_CLONE_DIR)

# 3) Installer dépendances (silencieux)
req = REPO_CLONE_DIR / "requirements.txt"
if req.exists():
    !pip -q install -r {str(req)}
else:
    print("⚠️ requirements.txt introuvable (pas bloquant).")

print("✅ Contenu repo (top-level):", [p.name for p in REPO_CLONE_DIR.iterdir()][:30])

# 4) Préparer data dir attendu PAR LE REPO cloné
REPO_DATA_DIR = REPO_CLONE_DIR / "data" / cfg.DATASET_ID
REPO_DATA_DIR.mkdir(parents=True, exist_ok=True)

# On copie (ou remplace) le dossier dataset dans le repo cloné
# => évite les erreurs de chemin lors du training
for f in ["train.parquet", "valid.parquet", "test.parquet", "item_seq.parquet", "item_info.parquet"]:
    src = DATA_READY_DIR / f
    assert src.exists(), f"Fichier manquant dans DATA_READY_DIR: {src}"
    shutil.copy(src, REPO_DATA_DIR / f)

print("✅ Data copiée dans repo cloné:", REPO_DATA_DIR)
print("   files:", [p.name for p in REPO_DATA_DIR.iterdir()])

# 5) Sanity check item_info format + colonne item_emb_d128
ii = pl.read_parquet(REPO_DATA_DIR / "item_info.parquet")
print("✅ item_info columns:", ii.columns)
assert "item_emb_d128" in ii.columns, "item_emb_d128 manquant !"

# Si jamais item_emb_d128 est list au lieu de array, on peut forcer array[Float64,128]
# (Optionnel mais recommandé)
try:
    # Si c'est list, on convertit vers Array fixe 128
    if "list" in str(ii.schema["item_emb_d128"]).lower():
        arr = np.array(ii["item_emb_d128"].to_list(), dtype=np.float64)
        assert arr.shape[1] == 128
        ii = ii.with_columns(
            pl.Series("item_emb_d128", arr.tolist()).cast(pl.Array(pl.Float64, 128))
        )
        ii.write_parquet(REPO_DATA_DIR / "item_info.parquet")
        print("✅ item_emb_d128 casté en Array(Float64,128)")
except Exception as e:
    print("ℹ️ Cast optionnel ignoré:", e)

print("✅ DONE 9.0")

✅ DATA_READY_DIR: /kaggle/working/WWW2025_MMCTR_Challenge/data/MicroLens_1M_x1
   files: ['valid.parquet', 'train.parquet', 'item_seq.parquet', 'item_info.parquet', 'test.parquet']
✅ Repo cloné dans: /kaggle/working/WWW2025_MMCTR_Challenge_repo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 46.7 MB/s eta 0:00:0000:0100:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.6.0 requires polars<1.29,>=1.25, but you have polars 1.0.0 which is incompatible.
✅ Contenu repo (top-level): ['requirements.txt', 'src', 'data', 'run_param_tuner.py', 'README.md', 'prediction.py', 'img', 'config', 'fuxictr_version.py', '.git', '.gitignore', 'LICENSE', 'run_expid.py']
✅ Data copiée dans repo clon

#### convertir proprement en float32

In [22]:
import polars as pl
import numpy as np
from pathlib import Path

DATA_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge/data/MicroLens_1M_x1")
path_in  = DATA_DIR / "item_info.parquet"

ii = pl.read_parquet(path_in)

# conversion list[f64] -> list[f32]
emb32 = [np.asarray(v, dtype=np.float32).tolist() for v in ii["item_emb_d128"].to_list()]
ii = ii.drop("item_emb_d128").with_columns(pl.Series("item_emb_d128", emb32))

# réécriture
ii.write_parquet(path_in)

print("✅ Re-saved in float32:", path_in)
print(ii.select(["item_id", "item_emb_d128"]).head(3))
print("Type:", ii.schema["item_emb_d128"])

✅ Re-saved in float32: /kaggle/working/WWW2025_MMCTR_Challenge/data/MicroLens_1M_x1/item_info.parquet
shape: (3, 2)
┌─────────┬─────────────────────────────────┐
│ item_id ┆ item_emb_d128                   │
│ ---     ┆ ---                             │
│ i64     ┆ list[f64]                       │
╞═════════╪═════════════════════════════════╡
│ 0       ┆ [0.0, 0.0, … 0.0]               │
│ 1       ┆ [-0.159443, 0.273246, … 0.0841… │
│ 2       ┆ [-0.221092, 0.021761, … 0.0078… │
└─────────┴─────────────────────────────────┘
Type: List(Float64)


## 9.1 Générer un YAML config (DIN / DCNv2 / AutoInt)
- Pour **Task1** : reste sur **DIN** et modifie seulement :
  - `learning_rate`, `batch_size`, `net_dropout`, `embedding_regularizer`, `net_regularizer`
- Pour **Task1&2** : tu peux changer le modèle + plus d’hyperparams.

> ⚠️ Le fichier YAML de base est dans :  
`config/MMCTR/{DATASET_ID}/...yaml`


In [26]:
# =========================================================
# 9.1) AUTO-FIND YAML TEMPLATE + GENERATE CONFIGS (ROBUST)
# =========================================================
from pathlib import Path
import yaml, copy

REPO_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge_repo")
assert REPO_DIR.exists(), f"Repo introuvable: {REPO_DIR}"
print("✅ REPO_DIR:", REPO_DIR)

ds_id = cfg.DATASET_ID  # "MicroLens_1M_x1"
data_root = str(REPO_DIR / "data")

# ---------------------------------------------------------
# 1) Trouver tous les YAML dans le repo
# ---------------------------------------------------------
yaml_files = sorted(list(REPO_DIR.rglob("*.yaml")) + list(REPO_DIR.rglob("*.yml")))
print("✅ Nb YAML trouvés:", len(yaml_files))
if len(yaml_files) == 0:
    raise FileNotFoundError("Aucun YAML trouvé dans le repo cloné. Vérifie /config dans le repo.")

# ---------------------------------------------------------
# 2) Priorité: YAML liés à MMCTR + dataset
# ---------------------------------------------------------
ds_candidates = [p for p in yaml_files if ds_id.lower() in str(p).lower()]
if len(ds_candidates) == 0:
    ds_candidates = [p for p in yaml_files if "mmctr" in str(p).lower()]
if len(ds_candidates) == 0:
    ds_candidates = yaml_files[:]  # fallback

print("✅ YAML candidates:", len(ds_candidates))

# ---------------------------------------------------------
# 3) Choisir un template DIN si possible
# ---------------------------------------------------------
def is_din_yaml(p: Path):
    return "din" in p.name.lower()

din_candidates = [p for p in ds_candidates if is_din_yaml(p)]
template_path = din_candidates[0] if len(din_candidates) > 0 else ds_candidates[0]
print("✅ Template choisi:", template_path)

# ---------------------------------------------------------
# 4) Load/Save YAML
# ---------------------------------------------------------
def load_yaml(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

def save_yaml(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        yaml.safe_dump(obj, f, sort_keys=False, allow_unicode=True)

base_yaml = load_yaml(template_path)

# ---------------------------------------------------------
# 5) Update keys recursively (robuste)
# ---------------------------------------------------------
def set_keys_recursive(obj, mapping):
    """
    mapping: dict key->value ; si la clé existe n'importe où dans la structure, on la remplace.
    """
    if isinstance(obj, dict):
        for k in list(obj.keys()):
            if k in mapping:
                obj[k] = mapping[k]
            else:
                set_keys_recursive(obj[k], mapping)
    elif isinstance(obj, list):
        for it in obj:
            set_keys_recursive(it, mapping)

def ensure_tuner_params(y):
    if "tuner_params" not in y or y["tuner_params"] is None or not isinstance(y["tuner_params"], dict):
        y["tuner_params"] = {}
    return y

# ---------------------------------------------------------
# 6) Générer configs (DIN Task1 / DCNv2 Task1&2)
# ---------------------------------------------------------
def make_config(model_name="DIN", task1_compliant=True):
    y = copy.deepcopy(base_yaml)

    # model name (si la clé existe)
    if isinstance(y, dict):
        y["model"] = model_name

    # mettre dataset_id + data_root où ça existe
    set_keys_recursive(y, {
        "dataset_id": ds_id,
        "data_root": data_root,
        "data_dir": data_root,
        "data_path": data_root
    })

    y = ensure_tuner_params(y)

    if task1_compliant:
        # ✅ Task1: uniquement hyperparams autorisés
        y["tuner_params"] = {
            "learning_rate": [1e-3, 5e-4],
            "batch_size": [2048, 4096],
            "net_dropout": [0.0, 0.1, 0.2],
            "embedding_regularizer": [0.0, 1e-7, 1e-6],
            "net_regularizer": [0.0, 1e-7, 1e-6],
        }
    else:
        tp = y["tuner_params"]
        tp["learning_rate"] = [1e-3, 5e-4, 2e-4]
        tp["batch_size"] = [1024, 2048, 4096]
        tp["net_dropout"] = [0.0, 0.1, 0.2, 0.3]
        tp["embedding_regularizer"] = [0.0, 1e-7, 1e-6, 1e-5]
        tp["net_regularizer"] = [0.0, 1e-7, 1e-6, 1e-5]

        if model_name.lower() in ["dcnv2", "autoint"]:
            tp["dnn_hidden_units"] = ["[512,256,128]", "[1024,512,256]"]

    return y

OUT_CFG_DIR = REPO_DIR / "config" / "CUSTOM"
task1_yaml_path  = OUT_CFG_DIR / f"DIN_{ds_id}_task1_aucboost.yaml"
task12_yaml_path = OUT_CFG_DIR / f"DCNv2_{ds_id}_task12_aucboost.yaml"

save_yaml(make_config("DIN", task1_compliant=True), task1_yaml_path)
save_yaml(make_config("DCNv2", task1_compliant=False), task12_yaml_path)

print("✅ Saved configs:")
print(" -", task1_yaml_path)
print(" -", task12_yaml_path)

✅ REPO_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo
✅ Nb YAML trouvés: 2
✅ YAML candidates: 2
✅ Template choisi: /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/DIN_microlens_mmctr_tuner_config_01.yaml
✅ Saved configs:
 - /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM/DIN_MicroLens_1M_x1_task1_aucboost.yaml
 - /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM/DCNv2_MicroLens_1M_x1_task12_aucboost.yaml


## 9.2 Training (param tuner)
Exécute **un** des deux :
- Task1 : `DIN_task1_aucboost.yaml`
- Task1&2 : `DCNv2_task12_aucboost.yaml` (ou change model_name)

> Kaggle GPU recommandé.


In [28]:
# =========================================================
# 9.x) FIX NumPy2 + PREP + RUN PARAM TUNER (FULL)
# =========================================================
import os, sys, gc, re, warnings, shutil
from pathlib import Path

warnings.filterwarnings("ignore")

# -------------------------
# 0) Paths (adapte si besoin)
# -------------------------
REPO_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge_repo")  # ton repo cloné
DATASET_ID = "MicroLens_1M_x1"                                  # cfg.DATASET_ID
DATA_DIR = REPO_DIR / "data" / DATASET_ID                       # data déjà copiée
CFG_PATH = REPO_DIR / "config" / "CUSTOM" / f"DCNv2_{DATASET_ID}_task12_aucboost.yaml"  # ton yaml custom
GPU_ID = 0

print("✅ REPO_DIR:", REPO_DIR)
print("✅ DATA_DIR:", DATA_DIR)
print("✅ CFG_PATH:", CFG_PATH)

assert REPO_DIR.exists(), f"Repo introuvable: {REPO_DIR}"
assert DATA_DIR.exists(), f"Data dir introuvable: {DATA_DIR}"
assert CFG_PATH.exists(), f"YAML config introuvable: {CFG_PATH}"

print("✅ files data:", sorted([p.name for p in DATA_DIR.iterdir() if p.is_file()]))

# =========================================================
# 1) FIX CRASH NumPy2: patch np.Inf -> np.inf dans FuxiCTR
# =========================================================
def patch_np_inf_in_fuxictr():
    """
    Patch fuxictr rank_model.py: np.Inf -> np.inf (NumPy 2.0 compat)
    """
    import fuxictr
    base = Path(fuxictr.__file__).parent
    target = base / "pytorch" / "models" / "rank_model.py"
    if not target.exists():
        print("⚠️ rank_model.py introuvable, on cherche dans site-packages...")
        # fallback search
        import site
        found = []
        for sp in site.getsitepackages():
            cand = Path(sp) / "fuxictr" / "pytorch" / "models" / "rank_model.py"
            if cand.exists():
                found.append(cand)
        if not found:
            raise FileNotFoundError("Impossible de trouver fuxictr/pytorch/models/rank_model.py")
        target = found[0]

    txt = target.read_text(encoding="utf-8")
    if "np.Inf" not in txt and "-np.Inf" not in txt:
        print("✅ Patch déjà OK (np.Inf absent):", target)
        return

    txt2 = txt.replace("np.Inf", "np.inf").replace("-np.Inf", "-np.inf")
    target.write_text(txt2, encoding="utf-8")
    print("✅ Patch appliqué:", target)
    print("✅ Vérif np.Inf encore présent ?", ("np.Inf" in target.read_text(encoding="utf-8")))

# Appliquer patch
patch_np_inf_in_fuxictr()

# =========================================================
# 2) (Optionnel) Vérification rapide versions
# =========================================================
import numpy as np
print("✅ numpy:", np.__version__)

# =========================================================
# 3) Lancer le training (param tuner)
# =========================================================
os.chdir(str(REPO_DIR))
print("📂 Current dir:", os.getcwd())

# Important: sur Kaggle parfois python pointe vers /usr/bin/python
# On utilise sys.executable pour être sûr.
cmd = f'"{sys.executable}" run_param_tuner.py --config "{CFG_PATH}" --gpu {GPU_ID}'
print("▶️ CMD:", cmd)

# Exécution
ret = os.system(cmd)
print("✅ finished with code:", ret)

# =========================================================
# 4) (Optionnel) Trouver le meilleur checkpoint produit
# =========================================================
ckpt_dir = REPO_DIR / "checkpoints"
if ckpt_dir.exists():
    # Affiche quelques dossiers / fichiers récents
    all_ckpt = sorted(ckpt_dir.rglob("*"), key=lambda p: p.stat().st_mtime, reverse=True)[:20]
    print("\n📦 Derniers éléments checkpoints:")
    for p in all_ckpt:
        print(" -", p.relative_to(REPO_DIR))
else:
    print("⚠️ Pas de dossier checkpoints trouvé.")

✅ REPO_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo
✅ DATA_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1
✅ CFG_PATH: /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM/DCNv2_MicroLens_1M_x1_task12_aucboost.yaml
✅ files data: ['feature_map.json', 'feature_processor.pkl', 'feature_vocab.json', 'item_info.parquet', 'item_seq.parquet', 'test.parquet', 'train.parquet', 'valid.parquet']
✅ Patch appliqué: /usr/local/lib/python3.12/dist-packages/fuxictr/pytorch/models/rank_model.py
✅ Vérif np.Inf encore présent ? False
✅ numpy: 2.0.2
📂 Current dir: /kaggle/working/WWW2025_MMCTR_Challenge_repo
▶️ CMD: "/usr/bin/python3" run_param_tuner.py --config "/kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM/DCNv2_MicroLens_1M_x1_task12_aucboost.yaml" --gpu 0


2025-12-19 00:38:42,703 P574 INFO FuxiCTR version: 2.3.7
2025-12-19 00:38:42,704 P574 INFO Params: {
    "accumulation_steps": "1",
    "attention_dropout": "0.1",
    "attention_hidden_activations": "ReLU",
    "attention_hidden_units": "[512, 256]",
    "attention_output_activation": "None",
    "batch_norm": "True",
    "batch_size": "8192",
    "data_format": "parquet",
    "data_root": "/kaggle/working/WWW2025_MMCTR_Challenge_repo/data",
    "dataset_id": "MicroLens_1M_x1",
    "debug_mode": "False",
    "din_use_softmax": "False",
    "dnn_activations": "ReLU",
    "dnn_hidden_units": "[1024, 512, 256]",
    "early_stop_patience": "3",
    "embedding_dim": "64",
    "embedding_regularizer": "1e-06",
    "epochs": "100",
    "eval_steps": "None",
    "feature_cols": "[{'active': True, 'dtype': 'int', 'name': 'user_id', 'type': 'meta'}, {'active': True, 'dtype': 'int', 'name': 'item_seq', 'type': 'meta'}, {'active': True, 'dtype': 'int', 'name': 'likes_level', 'type': 'categorical'

100%|█████████▉| 439/440 [08:10<00:01,  1.11s/it]

2025-12-19 00:47:07,220 P574 INFO Train loss: 0.134305
2025-12-19 00:47:07,220 P574 INFO Evaluation @epoch 1 - batch 440: 



100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


...


100%|██████████| 440/440 [08:16<00:00,  1.56s/it]

2025-12-19 06:34:57,821 P1014 INFO [Metrics] AUC: 0.888977
2025-12-19 06:34:57,822 P1014 INFO Monitor(max)=0.888977 STOP!
2025-12-19 06:34:57,822 P1014 INFO Reduce learning rate on plateau: 0.000001
2025-12-19 06:34:57,907 P1014 INFO ************ Epoch=25 end ************


 55%|█████▌    | 243/440 [04:36<03:43,  1.14s/it]


Traceback (most recent call last):
  File "/kaggle/working/WWW2025_MMCTR_Challenge_repo/run_expid.py", line 70, in <module>
    model.fit(train_gen, validation_data=valid_gen, **params)
  File "/usr/local/lib/python3.12/dist-packages/fuxictr/pytorch/models/rank_model.py", line 160, in fit
    self.train_epoch(data_generator)
  File "/usr/local/lib/python3.12/dist-packages/fuxictr/pytorch/models/rank_model.py", line 218, in train_epoch
    loss = self.train_step(batch_data)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/WWW2025_MMCTR_Challenge_repo/src/DIN.py", line 132, in train_step
    loss.backward()
  File "/usr/local/lib/python3.12/dist-packages/torch/_tensor.py", line 647, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.12/dist-packages/torch/autograd/__init__.py", line 354, in backward
    _engine_run_backward(
  File "/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py", line 829, in _engine_run_backward
    return Variable._e

Enumerate all tuner configurations done.
✅ finished with code: 0

📦 Derniers éléments checkpoints:
 - checkpoints/MicroLens_1M_x1/DIN_MicroLens_1M_x1_002_fb68e038.log
 - checkpoints/MicroLens_1M_x1/DIN_MicroLens_1M_x1_002_fb68e038.model
 - checkpoints/MicroLens_1M_x1
 - checkpoints/MicroLens_1M_x1/DIN_MicroLens_1M_x1_001_9377cbcb.log
 - checkpoints/MicroLens_1M_x1/DIN_MicroLens_1M_x1_001_9377cbcb.model


# 10) Inference + création du fichier submission
On cherche le checkpoint le plus récent, puis on prédit sur test et on écrit `prediction.csv`.


In [38]:
from pathlib import Path
import yaml

# 1) Chemins de base
REPO_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge_repo")
CONFIG_DIR = REPO_DIR / "config"
CUSTOM_DIR = CONFIG_DIR / "CUSTOM"

EXP_ID = "DIN_MicroLens_1M_x1_001_9377cbcb"  # ton meilleur modèle
print(f"📁 REPO_DIR: {REPO_DIR}")
print(f"🏷️ EXP_ID: {EXP_ID}")
print(f"📂 CUSTOM_DIR: {CUSTOM_DIR}")

# 2) Lister les YAML dispo dans config/CUSTOM
yaml_files = sorted(CUSTOM_DIR.glob("*.yaml"))
print("✅ YAML trouvés dans config/CUSTOM :")
for y in yaml_files:
    print("   -", y.name)

# 3) Chercher le bon fichier : DCNv2_MicroLens_1M_x1_task12_aucboost.yaml
CONFIG_PATH = None
for y in yaml_files:
    if "DCNv2_MicroLens_1M_x1_task12_aucboost.yaml" in str(y):
        CONFIG_PATH = y
        break

# Si tu veux être plus souple, tu peux utiliser juste "DCNv2_MicroLens_1M_x1_task12_aucboost"
# for y in yaml_files:
#     if "DCNv2_MicroLens_1M_x1_task12_aucboost" in y.name:
#         CONFIG_PATH = y
#         break

assert CONFIG_PATH is not None, "❌ Impossible de trouver le fichier DCNv2_MicroLens_1M_x1_task12_aucboost.yaml dans config/CUSTOM"
assert CONFIG_PATH.exists(), f"❌ Config introuvable sur le disque : {CONFIG_PATH}"

print(f"📄 CONFIG_PATH sélectionné : {CONFIG_PATH}")

# 4) Charger le YAML dans un dict Python
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

print("✅ YAML chargé. Clés au niveau racine :")
print(list(config.keys()))

# Si tu as besoin plus tard :
# model_params   = config.get("model", {})
# dataset_params = config.get("dataset", {})
# training_params = config.get("training", {})

📁 REPO_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo
🏷️ EXP_ID: DIN_MicroLens_1M_x1_001_9377cbcb
📂 CUSTOM_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM
✅ YAML trouvés dans config/CUSTOM :
   - DCNv2_MicroLens_1M_x1_task12_aucboost.yaml
   - DIN_MicroLens_1M_x1_task1_aucboost.yaml
📄 CONFIG_PATH sélectionné : /kaggle/working/WWW2025_MMCTR_Challenge_repo/config/CUSTOM/DCNv2_MicroLens_1M_x1_task12_aucboost.yaml
✅ YAML chargé. Clés au niveau racine :
['base_config', 'base_expid', 'dataset_id', 'dataset_config', 'tuner_space', 'model', 'tuner_params']


In [39]:
import pandas as pd
import numpy as np
from pathlib import Path

# ==========================
# 1. Paths de base
# ==========================
REPO_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge_repo")
DATA_DIR = REPO_DIR / "data" / "MicroLens_1M_x1"

print("📁 REPO_DIR:", REPO_DIR)
print("📁 DATA_DIR:", DATA_DIR)

# Candidats possibles pour le fichier train
train_candidates = [
    DATA_DIR / "train.parquet",
    DATA_DIR / "MicroLens_1M_x1_train.parquet"
]

TRAIN_PATH = None
for p in train_candidates:
    if p.exists():
        TRAIN_PATH = p
        break

if TRAIN_PATH is None:
    raise FileNotFoundError(
        f"Aucun fichier train.parquet trouvé dans {DATA_DIR}\n"
        "➡️ Adapte manuellement TRAIN_PATH en regardant le contenu du dossier."
    )

TEST_PATH = DATA_DIR / "test.parquet"
if not TEST_PATH.exists():
    raise FileNotFoundError(f"Fichier test.parquet introuvable: {TEST_PATH}")

print("✅ TRAIN_PATH:", TRAIN_PATH)
print("✅ TEST_PATH:", TEST_PATH)

# ==========================
# 2. Chargement des données
# ==========================
train_df = pd.read_parquet(TRAIN_PATH)
test_df = pd.read_parquet(TEST_PATH)

print("✅ train shape:", train_df.shape)
print("✅ test shape:", test_df.shape)
print("📎 Colonnes train:", list(train_df.columns))
print("📎 Colonnes test :", list(test_df.columns))

# ==========================
# 3. Détection automatique des colonnes
# ==========================

# 3.1 Colonne label
label_candidates = ["label", "target", "click", "y"]
LABEL_COL = None
for c in label_candidates:
    if c in train_df.columns:
        LABEL_COL = c
        break

if LABEL_COL is None:
    raise ValueError(
        "Impossible de trouver la colonne label. "
        f"Colonnes disponibles dans train: {list(train_df.columns)}\n"
        "➡️ Mets le vrai nom dans LABEL_COL."
    )

print(f"✅ Colonne label détectée: {LABEL_COL}")

# 3.2 Colonne item
item_candidates = ["item_id", "item", "product_id"]
ITEM_COL = None
for c in item_candidates:
    if c in train_df.columns:
        ITEM_COL = c
        break

if ITEM_COL is None:
    raise ValueError(
        "Impossible de trouver la colonne item.\n"
        "➡️ Mets manuellement le nom dans ITEM_COL (ex: 'item_id')."
    )

print(f"✅ Colonne item détectée: {ITEM_COL}")

# 3.3 Colonne id pour la soumission (optionnel mais pratique)
id_candidates = ["sample_id", "id", "row_id"]
ID_COL = None
for c in id_candidates:
    if c in test_df.columns:
        ID_COL = c
        break

if ID_COL is not None:
    print(f"✅ Colonne id détectée pour la soumission: {ID_COL}")
else:
    print("ℹ️ Aucune colonne id explicite détectée. "
          "On utilisera juste l'ordre des lignes pour prediction.csv.")

# ==========================
# 4. Calcul de la popularité (CTR)
# ==========================
print("📊 Calcul des CTR par item...")

group = train_df.groupby(ITEM_COL)[LABEL_COL].agg(["sum", "count"]).reset_index()
group.rename(columns={"sum": "clicks", "count": "impressions"}, inplace=True)

# CTR lissé (Laplace)
group["ctr"] = (group["clicks"] + 1.0) / (group["impressions"] + 2.0)

print("✅ Exemple de stats item :")
print(group.head())

# CTR global au cas où certains items de test n'apparaissent pas dans train
global_ctr = (train_df[LABEL_COL].sum() + 1.0) / (len(train_df) + 2.0)
print(f"🌍 CTR global (fallback) : {global_ctr:.6f}")

# ==========================
# 5. Jointure sur le test
# ==========================
print("🔗 Merge test_df avec les CTR des items...")

test_with_ctr = test_df.merge(group[[ITEM_COL, "ctr"]],
                              on=ITEM_COL,
                              how="left")

missing = test_with_ctr["ctr"].isna().sum()
print(f"ℹ️ Items de test sans stats dans le train : {missing}")

test_with_ctr["ctr"].fillna(global_ctr, inplace=True)

# ==========================
# 6. Construction du DataFrame de soumission
# ==========================
if ID_COL is not None:
    submission = pd.DataFrame({
        ID_COL: test_with_ctr[ID_COL],
        "prediction": test_with_ctr["ctr"].astype(float)
    })
else:
    # Pas d'id explicite -> juste une colonne 'prediction'
    submission = pd.DataFrame({
        "prediction": test_with_ctr["ctr"].astype(float)
    })

OUT_PATH = Path("/kaggle/working/prediction.csv")
submission.to_csv(OUT_PATH, index=False)

print("✅ prediction.csv généré :", OUT_PATH)
print(submission.head())

📁 REPO_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo
📁 DATA_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1
✅ TRAIN_PATH: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1/train.parquet
✅ TEST_PATH: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1/test.parquet
✅ train shape: (3600000, 6)
✅ test shape: (379142, 6)
📎 Colonnes train: ['user_id', 'item_seq', 'item_id', 'likes_level', 'views_level', 'label']
📎 Colonnes test : ['ID', 'user_id', 'item_seq', 'item_id', 'likes_level', 'views_level']
✅ Colonne label détectée: label
✅ Colonne item détectée: item_id
ℹ️ Aucune colonne id explicite détectée. On utilisera juste l'ordre des lignes pour prediction.csv.
📊 Calcul des CTR par item...
✅ Exemple de stats item :
   item_id  clicks  impressions       ctr
0        1       1          203  0.009756
1        2       1          216  0.009174
2        3       1          183  0.010811
3        4      17          193  0.092308
4        5    

In [40]:
import pandas as pd
import numpy as np
from pathlib import Path

# ==========================
# 1. Paths de base
# ==========================
REPO_DIR = Path("/kaggle/working/WWW2025_MMCTR_Challenge_repo")
DATA_DIR = REPO_DIR / "data" / "MicroLens_1M_x1"

TRAIN_PATH = DATA_DIR / "train.parquet"
TEST_PATH  = DATA_DIR / "test.parquet"

print("📁 REPO_DIR:", REPO_DIR)
print("📁 DATA_DIR:", DATA_DIR)
print("✅ TRAIN_PATH:", TRAIN_PATH)
print("✅ TEST_PATH :", TEST_PATH)

# ==========================
# 2. Chargement
# ==========================
train_df = pd.read_parquet(TRAIN_PATH)
test_df  = pd.read_parquet(TEST_PATH)

print("✅ train shape:", train_df.shape)
print("✅ test shape :", test_df.shape)
print("📎 Colonnes train:", list(train_df.columns))
print("📎 Colonnes test :", list(test_df.columns))

# On sait déjà, d'après ton log :
LABEL_COL = "label"
ITEM_COL  = "item_id"
ID_COL    = "ID"        # colonne ID imposée par le challenge

# ==========================
# 3. CTR baseline par item
# ==========================
print("📊 Calcul CTR par item...")

group = (
    train_df
    .groupby(ITEM_COL)[LABEL_COL]
    .agg(["sum", "count"])
    .reset_index()
    .rename(columns={"sum": "clicks", "count": "impressions"})
)

# Lissage Laplace pour éviter 0/1 extrêmes
group["ctr"] = (group["clicks"] + 1.0) / (group["impressions"] + 2.0)

print("✅ Exemple stats item :")
print(group.head())

# CTR global si item du test jamais vu dans train
global_ctr = (train_df[LABEL_COL].sum() + 1.0) / (len(train_df) + 2.0)
print(f"🌍 CTR global fallback : {global_ctr:.6f}")

# ==========================
# 4. Join avec le test
# ==========================
print("🔗 Merge test + CTR...")

test_with_ctr = test_df.merge(
    group[[ITEM_COL, "ctr"]],
    on=ITEM_COL,
    how="left"
)

missing = test_with_ctr["ctr"].isna().sum()
print(f"ℹ️ Items test sans stats dans train : {missing}")

test_with_ctr["ctr"].fillna(global_ctr, inplace=True)

# ==========================
# 5. Construire prediction.csv
# ==========================
# Ici on participe à Task2 -> colonnes: ID, Task2
submission = pd.DataFrame({
    "ID": test_with_ctr[ID_COL],
    "Task2": test_with_ctr["ctr"].astype(float)
})

OUT_PATH = Path("/kaggle/working/prediction-task2.csv")
submission.to_csv(OUT_PATH, index=False)

print("✅ prediction.csv généré :", OUT_PATH)
print(submission.head())

📁 REPO_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo
📁 DATA_DIR: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1
✅ TRAIN_PATH: /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1/train.parquet
✅ TEST_PATH : /kaggle/working/WWW2025_MMCTR_Challenge_repo/data/MicroLens_1M_x1/test.parquet
✅ train shape: (3600000, 6)
✅ test shape : (379142, 6)
📎 Colonnes train: ['user_id', 'item_seq', 'item_id', 'likes_level', 'views_level', 'label']
📎 Colonnes test : ['ID', 'user_id', 'item_seq', 'item_id', 'likes_level', 'views_level']
📊 Calcul CTR par item...
✅ Exemple stats item :
   item_id  clicks  impressions       ctr
0        1       1          203  0.009756
1        2       1          216  0.009174
2        3       1          183  0.010811
3        4      17          193  0.092308
4        5       3          198  0.020000
🌍 CTR global fallback : 0.250000
🔗 Merge test + CTR...
ℹ️ Items test sans stats dans train : 7251
✅ prediction.csv généré : /kaggle/working/pr